In [52]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from dataclasses import dataclass
import math

In [26]:
@dataclass
class GPTConfig:
    block_size: int = 1024
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    vocab_size: int = 50257

In [27]:
class CasualSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))
    def forward(self, x):
        B, T, C = x.shape
        qkv = self.c_attn(x) # B, T, 3 * C
        q, k, v = qkv.split(self.n_embd, dim=2)
        q = q.view(B, T, self.n_head, C//self.n_head).transpose(1,2) # B, nh, T, hs
        k = k.view(B, T, self.n_head, C//self.n_head).transpose(1,2) # B, nh, T, hs
        v = v.view(B, T, self.n_head, C//self.n_head).transpose(1,2) # B, nh, T, hs
        wei = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))  # # B, nh, T, T
        wei = wei.masked_fill(self.bias[:,:,:T,:T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1) 
        out = wei @ v # B, nh, T, hs
        return self.c_proj(out.transpose(1, 2).contiguous().view(B, T, C))
        

In [28]:
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, config.n_embd * 4)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(config.n_embd * 4, config.n_embd)
    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

In [29]:
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CasualSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)
    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

In [30]:
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = nn.ModuleDict(dict(
        wte = nn.Embedding(config.vocab_size, config.n_embd),
        wpe = nn.Embedding(config.block_size, config.n_embd),
        h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
        ln_f = nn.LayerNorm(config.n_embd)))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
    def forward(self, idx):
        B, T = idx.shape
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # T
        pos_embd = self.transformer.wpe(pos) # T, C
        tok_embd = self.transformer.wte(idx) # B, T, C
        x = pos_embd + tok_embd # B, T, C
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        return logits
            
    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

In [32]:
model = GPT.from_pretrained('gpt2')
print("didn't crash yay!")

loading weights from pretrained gpt: gpt2
didn't crash yay!


In [44]:
model.eval()
max_length = 30
num_return_sequences = 5

import tiktoken
enc = tiktoken.get_encoding('gpt2')
token = enc.encode("Hello, I'm a language model,")
token = torch.tensor(token, dtype=torch.long)
token = token.unsqueeze(0).repeat(num_return_sequences, 1)

In [57]:
x = token.clone()
print(x.shape)
while x.shape[1] < max_length:
    with torch.no_grad():
        logits = model(x)
        logits = logits[:, -1, :] # (B, vocab_size)
        probs = F.softmax(logits, -1)
        topK, topKInd = torch.topk(probs, 50)
        ix = torch.multinomial(topK, num_samples=1)
        ids = torch.gather(topKInd, dim=-1, index=ix)
        x = torch.cat((x, ids), dim=1)

torch.Size([5, 8])


In [62]:
for i in range(num_return_sequences):
    tokens = x[i].tolist()
    print(enc.decode(tokens))
    print("\n")

Hello, I'm a language model, so I don't take them all. Instead I think in this context, I find them helpful and useful in


Hello, I'm a language model, I'm a framework, and I've never heard of this."

A recent Harvard thesis, "C


Hello, I'm a language model, one with a lot of structure, a lot of variables and it's a bit cumbersome to read, so I


Hello, I'm a language model, let's start with a simple one.

A language model means the model is as simple as possible for


Hello, I'm a language model, a model that we could all live with. We built the whole thing in languages like C++, Perl.


